In [1]:
import os
import time
import numpy as np
from glob import glob
from tqdm.auto import tqdm
import matplotlib.pyplot as plt

import nrrd
import tifffile
import nibabel as nib
from medpy.io import load as medload
from medpy.io import save as medsave
from skimage.io import imread as skimread

In [10]:
f_tiff = 'sample_files/sample.tiff'
f_nrrd = 'sample_files/sample.nrrd'
f_nii = 'sample_files/sample.nii'
f_npy = 'sample_files/sample.npy'

## save a random image

In [ ]:
ext = '.tiff'
for i in range(1,11):
  fname = f'sample_files/sample_tiffs3d/sample{i}{ext}'
  img = np.random.randint(256, size=(16,32,64), dtype=np.uint8)

  if ext == '.tiff':
    tifffile.imwrite(fname,img)
  elif ext == '.npy':
    np.save(fname, img)
  else:
    medsave(img, fname)

In [104]:
img = np.random.randint(256, size=(2056,1024,1024), dtype=np.uint8)

In [11]:
tifffile.imwrite(f_tiff, img)
medsave(img, f_nrrd)
medsave(img, f_nii)
np.save(f_npy, img)

## Compare loading times

### tiff readers

In [60]:
%timeit -r 100 tifffile.imread(f_tiff)

1.44 s ± 14.1 ms per loop (mean ± std. dev. of 100 runs, 1 loop each)


In [61]:
%timeit -r 100 medload(f_tiff)

2.45 s ± 35.7 ms per loop (mean ± std. dev. of 100 runs, 1 loop each)


In [62]:
%timeit -r 100 skimread(f_tiff)

1.44 s ± 13 ms per loop (mean ± std. dev. of 100 runs, 1 loop each)


### nrrd readers

In [65]:
%timeit -r 100 nrrd.read(f_nrrd)

950 ms ± 3.86 ms per loop (mean ± std. dev. of 100 runs, 1 loop each)


In [66]:
%timeit -r 100 medload(f_nrrd)

2.53 s ± 30.7 ms per loop (mean ± std. dev. of 100 runs, 1 loop each)


In [67]:
%timeit -r 100 skimread(f_nrrd)

2.54 s ± 32.4 ms per loop (mean ± std. dev. of 100 runs, 1 loop each)


### nii readers

In [40]:
%timeit -r 100 nib.load(f_nii)

450 µs ± 5.39 µs per loop (mean ± std. dev. of 100 runs, 1000 loops each)


In [33]:
%timeit -r 100 np.array(nib.load(f_nii).dataobj)

893 ms ± 1.71 ms per loop (mean ± std. dev. of 100 runs, 1 loop each)


In [68]:
%timeit -r 100 medload(f_nii)

3.54 s ± 30.7 ms per loop (mean ± std. dev. of 100 runs, 1 loop each)


In [69]:
%timeit -r 100 skimread(f_nii)

3.54 s ± 28.8 ms per loop (mean ± std. dev. of 100 runs, 1 loop each)


In [41]:
%timeit -r 100 np.array(nib.load(f_nii).get_fdata())

10.1 s ± 64.3 ms per loop (mean ± std. dev. of 100 runs, 1 loop each)


## check reader

In [1]:
import univread

In [2]:
from importlib import reload
reload(univread)

<module 'univread' from '/home/ws/er5241/Repos/UnivRead3D/univread/__init__.py'>

In [3]:
img = univread.read('sample_files/sample_tiffs3d')
print(type(img))
print(img.shape)

<class 'numpy.ndarray'>
(160, 32, 64)


In [4]:
# check .nii.gz sample

img = univread.read('sample_files/sample.nii.gz')
print(type(img))
print(img.shape)

<class 'numpy.ndarray'>
(64, 32, 16)


In [5]:
# check unknown sample

img = univread.read('sample_files/sample.mha')
print(type(img))
print(img.shape)

<class 'numpy.ndarray'>
(64, 32, 16)


/home/ws/er5241/Repos/UnivRead3D/univread/univ_read.py:44: UserWarning: unrecognized file extension, proceeding to load with medpy
  warnings.warn('unrecognized file extension, proceeding to load with medpy')


In [4]:
# check if all match

a,b,c = univread.read(f_tiff), univread.read(f_nii), univread.read(f_nrrd)
d,e = univread.read('sample_files/sample.nii.gz'), univread.read('sample_files/sample.npy')

print((a==b).any(), (b==c).any(), (c==d).any(), (d==e).any())

True True True True


## check mem-mapping

In [8]:
%load_ext memory_profiler

### tiff

In [23]:
%memit univread.read(f_tiff, lazy=False)

peak memory: 4222.41 MiB, increment: 1969.08 MiB


In [17]:
%memit univread.read(f_tiff, lazy=True)

peak memory: 2253.32 MiB, increment: 0.20 MiB


In [26]:
print(type(univread.read(f_tiff, lazy=False)))
print(type(univread.read(f_tiff, lazy=True)))

<class 'numpy.ndarray'>
<class 'numpy.memmap'>


### nii

In [24]:
%memit univread.read(f_nii, lazy=False)

peak memory: 5932.89 MiB, increment: 3679.56 MiB


In [25]:
%memit univread.read(f_nii, lazy=True)

peak memory: 2253.39 MiB, increment: 0.00 MiB


In [5]:
print(type(univread.read(f_nii, lazy=False)))
print(type(univread.read(f_nii, lazy=True)))

<class 'numpy.ndarray'>
<class 'numpy.memmap'>


### npy

In [12]:
%memit univread.read(f_npy, lazy=False)

peak memory: 2197.21 MiB, increment: 1995.72 MiB


In [13]:
%memit univread.read(f_npy, lazy=True)

peak memory: 201.55 MiB, increment: 0.00 MiB


In [14]:
print(type(univread.read(f_npy, lazy=False)))
print(type(univread.read(f_npy, lazy=True)))

<class 'numpy.ndarray'>
<class 'numpy.memmap'>


### dir containing tiffs

In [5]:
%memit univread.read("/home/ws/er5241/Repos/UnivRead3D/sample_files/sample_tiffs3d/", lazy=False)

peak memory: 16578.19 MiB, increment: 16367.24 MiB


In [6]:
%memit univread.read("/home/ws/er5241/Repos/UnivRead3D/sample_files/sample_tiffs3d/", lazy=True)

peak memory: 211.87 MiB, increment: 1.35 MiB


In [7]:
x1 = univread.read("/home/ws/er5241/Repos/UnivRead3D/sample_files/sample_tiffs3d/", lazy=False)
x2 = univread.read("/home/ws/er5241/Repos/UnivRead3D/sample_files/sample_tiffs3d/", lazy=True)

In [8]:
print(x1.shape)
print(x2.shape)
print((x1==x2).all())

(8224, 1024, 1024)
(8224, 1024, 1024)
True


### multi-page tiff

In [9]:
%memit univread.read('/mnt/HD-LSDF/Medaka/201912_beamtime_medaka/Medaka_1422_140-3/slices.tif', lazy=False)

peak memory: 220293.99 MiB, increment: 203634.88 MiB


In [5]:
%memit univread.read('/mnt/HD-LSDF/Medaka/201912_beamtime_medaka/Medaka_1422_140-3/slices.tif', lazy=True)

peak memory: 215.26 MiB, increment: 32.51 MiB


In [6]:
x1 = univread.read('/mnt/HD-LSDF/Medaka/201912_beamtime_medaka/Medaka_1422_140-3/slices.tif', lazy=False)
x2 = univread.read('/mnt/HD-LSDF/Medaka/201912_beamtime_medaka/Medaka_1422_140-3/slices.tif', lazy=True)

In [7]:
print(x1.shape)
print(x2.shape)
print((x1==x2).all())

(6571, 2016, 2016)
(6571, 2016, 2016)
True
